In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import json
from typing import Dict, List, Tuple

class NepaliEcommerceBot:
    def __init__(self):
        self.intents = {
            "greeting": [
                "namaste", "namaskar", "k cha", "kasto chha",
                "subha din", "subha prabhat"
            ],
            "order_status": [
                "mero order kahaa cha",
                "order kati din ma aaucha",
                "delivery status k cha",
                "mero parcel kahaa pugyo"
            ],
            "price_inquiry": [
                "kati parcha",
                "yo product ko dam kati ho",
                "price kati ho",
                "rate bhanna saknu huncha"
            ],
            "product_availability": [
                "stock ma cha",
                "aile uplabdha cha",
                "paaincha aile",
                "kata bata kinna sakincha"
            ]
        }

        self.responses = {
            "greeting": [
                "Namaste! Ma tapaailaai kasari sahayog garna sakchu?",
                "Swagat cha! Tapaailaai k sahayog chahincha?"
            ],
            "order_status": [
                "Tapaaiko order track garna, kripaya order ID provide garnus.",
                "Order status check garna malai order number chaincha."
            ],
            "price_inquiry": [
                "Kun product ko price chahincha? Product code wa naam bhannus.",
                "Product ko naam wa code provide garnus, ma price check gardinchu."
            ],
            "product_availability": [
                "Kun product ko availability check garna chahanu huncha?",
                "Product ko naam wa code bhannus, ma stock check gardinchu."
            ]
        }

        self.context_manager = ContextManager()
        self.vectorizer = None
        self.intent_vectors = None
        self._initialize_vectorizer()

    def _initialize_vectorizer(self):
        # Prepare training data
        all_intents = []
        intent_labels = []
        for intent, phrases in self.intents.items():
            all_intents.extend(phrases)
            intent_labels.extend([intent] * len(phrases))

        # Initialize and fit vectorizer
        self.vectorizer = TfidfVectorizer(ngram_range=(1, 3))
        self.intent_vectors = self.vectorizer.fit_transform(all_intents)

        # Save the model and vectorizer
        self._save_model()

    def _save_model(self):
        with open('chatbot_model.pkl', 'wb') as f:
            pickle.dump({
                'vectorizer': self.vectorizer,
                'intent_vectors': self.intent_vectors,
                'intents': self.intents,
                'responses': self.responses
            }, f)

    def _load_model(self):
        try:
            with open('chatbot_model.pkl', 'rb') as f:
                data = pickle.load(f)
                self.vectorizer = data['vectorizer']
                self.intent_vectors = data['intent_vectors']
                self.intents = data['intents']
                self.responses = data['responses']
        except FileNotFoundError:
            print("Model file not found. Initializing new model.")
            self._initialize_vectorizer()

    def predict_intent(self, user_input: str) -> Tuple[str, float]:
        # Vectorize user input
        input_vector = self.vectorizer.transform([user_input])

        # Calculate similarities
        similarities = cosine_similarity(input_vector, self.intent_vectors)

        # Get the highest similarity score and index
        max_similarity = np.max(similarities)
        if max_similarity < 0.3:  # Threshold for minimum confidence
            return "unknown", 0.0

        # Find the corresponding intent
        max_idx = np.argmax(similarities)
        intent_idx = max_idx % len(self.intents)
        intent = list(self.intents.keys())[intent_idx]

        return intent, max_similarity

    def get_response(self, user_input: str) -> str:
        # Get intent and confidence
        intent, confidence = self.predict_intent(user_input)

        # Update context with new intent
        self.context_manager.update_context(intent)

        # Get response based on intent and context
        if intent == "unknown":
            return "Maaf garnuhos, ma tapaaiko prasna bujhina. Kripaya arko tarika le sodhnus."

        # Get contextual response
        responses = self.responses[intent]
        return np.random.choice(responses)

class ContextManager:
    def __init__(self):
        self.context_history = []
        self.max_context_length = 5

    def update_context(self, intent: str):
        self.context_history.append(intent)
        if len(self.context_history) > self.max_context_length:
            self.context_history.pop(0)

    def get_current_context(self) -> str:
        if not self.context_history:
            return "new_conversation"
        return self.context_history[-1]

    def clear_context(self):
        self.context_history = []

# Example usage
def main():
    chatbot = NepaliEcommerceBot()

    print("Namaste! Ma tapaailaai kasari sahayog garna sakchu? (quit lekhnus band garna)")

    while True:
        user_input = input("Tapai: ")
        if user_input.lower() == 'quit':
            print("Dhanyabaad! Feri bhetaula!")
            break

        response = chatbot.get_response(user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()

Namaste! Ma tapaailaai kasari sahayog garna sakchu? (quit lekhnus band garna)
Tapai: malai product ko paisa bujhnu thiyo
Bot: Product ko naam wa code bhannus, ma stock check gardinchu.
Tapai: tshirt
Bot: Maaf garnuhos, ma tapaaiko prasna bujhina. Kripaya arko tarika le sodhnus.
Tapai: malai payment method bujhnu thiyo
Bot: Maaf garnuhos, ma tapaaiko prasna bujhina. Kripaya arko tarika le sodhnus.
Tapai: mero order ko barema bujhnu thiyo
Bot: Product ko naam wa code provide garnus, ma price check gardinchu.
Tapai: ORD123
Bot: Maaf garnuhos, ma tapaaiko prasna bujhina. Kripaya arko tarika le sodhnus.
Tapai: malai tshirt ko price bujhnu thiyp
Bot: Swagat cha! Tapaailaai k sahayog chahincha?
Tapai: namaskar
Bot: Order status check garna malai order number chaincha.
Tapai: hello
Bot: Maaf garnuhos, ma tapaaiko prasna bujhina. Kripaya arko tarika le sodhnus.
Tapai: hello
Bot: Maaf garnuhos, ma tapaaiko prasna bujhina. Kripaya arko tarika le sodhnus.
Tapai: namaste
Bot: Swagat cha! Tapaailaai